## Number of COVID Cases and Deaths by Sex and Age

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [9]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
from uk_covid19 import Cov19API
import json

filters = [
    'areaType=nation',
    'areaName=England'
]
structure = {
    "males": "maleCases",
    "maledeaths": "maleDeaths28Days",
    "females": "femaleCases",
    "femaledeaths": "femaleDeaths28Days"
}

api = Cov19API(filters=filters, structure=structure)

GenAgeDistribution=api.get_json()

with open("GenAgeDistribution", "wt") as OUTF:
    json.dump(GenAgeDistribution, OUTF)

In [15]:
def wrangle_data(rawdata):
    with open("GenAgeDistribution", "rt") as INFILE:
        data=json.load(INFILE)
        
    datadic=data['data'][0]
    males=datadic['males']
    females=datadic['females']
    maledeaths=datadic["maledeaths"]
    femaledeaths=datadic["femaledeaths"]
    
    ageranges=[x['age'] for x in males]
    
    def min_age(agerange):
        agerange=agerange.replace('+','') # remove the + from 90+
        start=agerange.split('_')[0]
        return int(start)
    ageranges.sort(key=min_age)
    
    df=pd.DataFrame(index=ageranges, columns=['males', 'maledeaths','females', 'femaledeaths'])
    
    for entry in males: 
        ageband=entry['age'] 
        df.loc[ageband, 'males']=entry['value']
    for entry in maledeaths: 
        ageband=entry['age'] 
        df.loc[ageband, 'maledeaths']=entry['value']
    for entry in females:
        ageband=entry['age']
        df.loc[ageband, 'females']=entry['value']
    
    for entry in femaledeaths:
        ageband=entry['age']
        df.loc[ageband, 'femaledeaths']=entry['value']
    return df

GenAge_df=wrangle_data(GenAgeDistribution)

Please click "Refresh Data" to ensure that the graphs below have the most up to date data from the Public Health of England. 

In [18]:
def access_api():
    from uk_covid19 import Cov19API
    import json

    filters = [
    'areaType=nation',
    'areaName=England'
]
    structure = {
    "males": "maleCases",
    "maledeaths": "maleDeaths28Days",
    "females": "femaleCases",
    "femaledeaths": "femaleDeaths28Days"
}

    api = Cov19API(filters=filters, structure=structure)

    GenAgeDistribution=api.get_json()
    return GenAgeDistribution

In [19]:
def api_button_callback(button):
    apidata=access_api()
    global df
    df=wrangle_data(GenAgeDistribution)
    apibutton.icon="check"
    apibutton.disabled=True
        
apibutton=wdg.Button(
    description='Refresh Data', # you may want to change this...
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click to download current Public Health England data",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='download'
)

apibutton.on_click(api_button_callback)
display(apibutton)

Button(button_style='info', description='Refresh Data', icon='download', style=ButtonStyle(), tooltip='Click t…

## Number of COVID cases in Males and Females by age group in England

The below graph shows the number of COVID cases seperated by Males and Females in the age groups from 0 to 90+. The data presented only includes cases in England. To see data for each sex seperately, please select males or females in the sex category. To compare male and female cases, please hold control and click on both males and females. 

In [24]:
def GenAge_Plot(graphcolumns):
    ncols=len(graphcolumns)
    if ncols>0:
        GenAge_df.plot(kind='bar', y=list(graphcolumns)) # graphcolumns is a tuple - we need a list
        plt.show() # important - graphs won't update properly if this is missing
    else:
        # if the user has not selected any column, print a message instead
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

GenAgecols=wdg.SelectMultiple(
    options=['males', 'females'], # options available
    value=['males', 'females'], # initial value
    rows=2, # rows of the selection box
    description='sex',
    disabled=False
)

output=wdg.interactive_output(GenAge_Plot, {'graphcolumns': GenAgecols})

display(GenAgecols, output)


SelectMultiple(description='sex', index=(0, 1), options=('males', 'females'), rows=2, value=('males', 'females…

Output()

## Number of COVID Deaths in Males and Females by age group in England

The below graph shows the number of COVID deaths seperated by Males and Females in the age groups from 0 to 90+. The data presented only includes COVID deaths in England. To see data for each sex seperately, please select maledeaths or femaledeaths in the sex category. To compare male and female Covid deaths, please hold control and click on both maledeaths and femaledeaths. 

In [25]:
GenDeathAgecols=wdg.SelectMultiple(
    options=['maledeaths', 'femaledeaths'], # options available
    value=['maledeaths', 'femaledeaths'], # initial value
    rows=2, # rows of the selection box
    description='sex',
    disabled=False
)

output=wdg.interactive_output(GenAge_Plot, {'graphcolumns': GenDeathAgecols})

display(GenDeathAgecols, output)

SelectMultiple(description='sex', index=(0, 1), options=('maledeaths', 'femaledeaths'), rows=2, value=('malede…

Output()

**Author and Copyright Notice** This dashboard was created by Zainab Haybe *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*